# Collection and Cleaning of California Wildfire Data

---

What's the correlation between these wildfires and meteorilogical data? (avg temp, avg rainfall, co2 emissions). Plot average rainfall and average temperature linearly (x=time), using clusters of dots to show when fires occurred (the sizes representing acres burned).  

Training K-means clustering on latitude and longitude for locating hotspots. Potentially predicting future wildfires based on preceding meteorilogical data.
Training K-nearest neighbors on weather conditions to predict conditions that lead to fires.

In [43]:
county_ids = {
  'Tuolumne' : 'CA-109',
  'Los Angeles' : 'CA-037', 
  'Riverside' : 'CA-065', 
  'Placer' : 'CA-061', 
  'Ventura' : 'CA-111',
  'Fresno' : 'CA-019', 
  'Siskiyou' : 'CA-093', 
  'Humboldt' : 'CA-023',
  'Tehama' : 'CA-103', 
  'Shasta' : 'CA-089', 
  'San Diego' : 'CA-073',
  'Kern' : 'CA-029', 
  'Sonoma' : 'CA-097', 
  'Contra Costa' : 'CA-013', # added to SCU
  'Butte' : 'CA-007',
  'Tulare' : 'CA-107',
  'Santa Barbara' : 'CA-083', 
  'Mariposa' : 'CA-043', 
  'Monterey' : 'CA-053', 
  'El Dorado' : 'CA-017',
  'San Bernardino' : 'CA-071', 
  'Plumas' : 'CA-063', # PNF (Plumas National Forest)
  'Modoc' : 'CA-049', 
  'San Luis Obispo' : 'CA-079', 
  'Madera' : 'CA-039',
  'Inyo' : 'CA-027', # Inyo National Forest and Death Valley National Park (negligible)
  'Napa' : 'CA-055', 
  'San Benito' : 'CA-069', 
  'San Joaquin' : 'CA-077', # added to TCU
  'Lake' : 'CA-033', 
  'Alameda' : 'CA-001', # added to SCU
  'Glenn' : 'CA-021', 
  'Yolo' : 'CA-113', # added to LNU
  'Sacramento' : 'CA-067', # added to AEU
  'Stanislaus' : 'CA-099', # STF (Stanislaus National Forest)
  'Solano' : 'CA-095', # added to LNU
  'Merced' : 'CA-047',
  'Mendocino' : 'CA-045', 
  'Lassen' : 'CA-035', 
  'Amador' : 'CA-005', 
  'Yuba' : 'CA-115', 
  'Nevada' : 'CA-057', 
  'Santa Clara' : 'CA-085',
  'Calaveras' : 'CA-009', 
  'San Mateo' : 'CA-081', 
  'Orange' : 'CA-059', 
  'Colusa' : 'CA-011', # added to LNU
  'Trinity' : 'CA-105',
  'Del Norte' : 'CA-015', 
  'Mono' : 'CA-051', # Nothing exists here
  'Alpine' : 'CA-003', # Added to AEU
  'Sutter' : 'CA-101', # Added to NEU
  'Kings' : 'CA-031', 
  'Sierra' : 'CA-091', # Added to NEU
  'Santa Cruz' : 'CA-087', 
  'Marin' : 'CA-041', 
  'Mexico' : None, 
  'State of Oregon' : None,
  'State of Nevada' : None
}

In [44]:
unit_id_to_counties = {
  'AEU': ['Amador', 'El Dorado', 'Sacramento', 'Alpine'],
  'BDU': ['San Bernardino'],
  'BEU': ['San Benito', 'Monterey'],
  'BTU': ['Butte'],
  'CZU': ['San Mateo', 'Santa Cruz'], # south bay
  'FKU': ['Fresno', 'Kings'],
  'HUU': ['Humboldt', 'Del Norte'],
  'LMU': ['Lassen', 'Modoc'],
  'LNU': ['Sonoma', 'Lake', 'Napa', 'Yolo', 'Solano', 'Colusa'], # close fires
  'MEU': ['Mendocino'],
  'MMU': ['Madera', 'Mariposa', 'Merced'],
  'MVU': ['San Diego'],
  'NEU': ['Nevada', 'Yuba', 'Placer', 'Sutter', 'Sierra'],
  'RRU': ['Riverside'],
  'SCU': ['Santa Clara', 'Contra Costa', 'Alameda'], # east bay
  'SHU': ['Shasta', 'Trinity'],
  'SKU': ['Siskiyou'],
  'SLU': ['San Luis Obispo'],
  'TCU': ['Tuolumne', 'Calaveras', 'San Joaquin'],
  'TGU': ['Tehama', 'Glenn'],
  'TUU': ['Tulare'],
  'KRN': ['Kern'],
  'LAC': ['Los Angeles'],
  'MRN': ['Marin'],
  'ORC': ['Orange'],
  'SBC': ['Santa Barbara'],
  'VNC': ['Ventura'],
  'PNF': ['Plumas'],
  'STF': ['Stanislaus']
}

In [45]:
import pandas as pd
from altair import *
import requests
import time
import numpy as np
import copy

df_fire = pd.read_csv("ca_fires.csv")

df_fire = df_fire[df_fire["STATE"]=="CA"] # filter to CA only fires
df_fire["YEAR_"] = df_fire["YEAR_"].dropna().astype(int) # not working why?????
df_fire = df_fire.drop(df_fire[df_fire["YEAR_"]<=1900].index) # filter out everything before 1900
df_fire["ALARM_DATE"] = df_fire["ALARM_DATE"].map(lambda x: str(x)[:10])
df_fire["counties"] = df_fire["UNIT_ID"].map(unit_id_to_counties) # create county column

#df_fire["ALARM_DATE"] = pd.to_datetime(df_fire["ALARM_DATE"])

In [46]:
def climate_data(county):
  # get precip
  resp = requests.get('https://www.ncdc.noaa.gov/cag/county/time-series/%s-pcp-12-12-1900-2022.json?base_prd=true&begbaseyear=1901&endbaseyear=2000' % county_ids[county])
  df_county_precip = pd.DataFrame(resp.json()["data"]).T
  df_county_precip = df_county_precip.rename(columns={"value" : "precip_value", "anomaly" : "precip_anomaly"})
  time.sleep(0.5)
  # get temp
  resp = requests.get('https://www.ncdc.noaa.gov/cag/county/time-series/%s-tavg-12-12-1900-2022.json?base_prd=true&begbaseyear=1901&endbaseyear=2000' % county_ids[county])
  df_county_temp = pd.DataFrame(resp.json()["data"]).T
  df_county_temp = df_county_temp.rename(columns={"value" : "temp_value", "anomaly" : "temp_anomaly"})
  df_county = pd.concat([df_county_temp, df_county_precip], axis=1)
  # cast values to numeric
  df_county["temp_value"] = df_county["temp_value"].astype(float)
  df_county["temp_anomaly"] = df_county["temp_anomaly"].astype(float)
  df_county["precip_value"] = df_county["precip_value"].astype(float)
  df_county["precip_anomaly"] = df_county["precip_anomaly"].astype(float)
  # indices to year
  df_county = df_county.reset_index().drop("index", axis=1)
  df_county.index = df_county.index + 1900
  return df_county

In [47]:
# combines county data, getting average between them 

def combine_counties(unit_id):
  # combine counties in unit_id
  counties = copy.deepcopy(unit_id_to_counties.get(unit_id))
  length = len(counties)
  df_climate = climate_data(counties[0])
  counties.pop(0)
  while counties != []:
    time.sleep(0.5)
    temp = climate_data(counties[0])
    counties.pop(0)
    df_climate = df_climate + temp
  df_climate = df_climate / length
  return df_climate

In [48]:
def combine_data(df_fire, unit_id):
  # get fire data
  df_fire = df_fire.drop(df_fire[df_fire["UNIT_ID"] != unit_id].index)
  frequency = df_fire["YEAR_"].value_counts().sort_index()
  acres_sum = df_fire.groupby("YEAR_").sum()["GIS_ACRES"].sort_index()
  acres_mean = df_fire.groupby("YEAR_").mean()["GIS_ACRES"].sort_index()
  fire_data = pd.DataFrame({'frequency' : frequency, 'acres_sum' : acres_sum, 'acres_mean' : acres_mean})
  # combine with county data
  df_county = combine_counties(unit_id)
  return df_county.merge(fire_data, left_index=True, right_index=True, how='outer').fillna(0).reset_index().rename(columns={"index": "year"})

In [49]:
df_fire_counts = pd.DataFrame(df_fire["YEAR_"].value_counts().sort_index()).reset_index()
df_fire_counts["index"] = df_fire_counts["index"].astype(str).map(lambda x: x[:4]+'/12/31')

In [50]:
# lag = number of previous years to take into account

def precip_agg_mean(df, lag):
  temp = df["precip_value"]
  for i in range(1, lag+1):
    temp = temp + df["precip_value"].shift(i)
  temp = temp / (lag+1)
  return temp

In [51]:
def lag_correlation(df):
  lag_corr = []
  for i in range(10):
    df["precip_agg_mean"] = precip_agg_mean(df, i)
    lag_corr.append([i, df["frequency"].corr(df["precip_agg_mean"])])
  return pd.DataFrame(lag_corr, columns=['lag', 'r2'])

In [52]:
df_lnu = combine_data(df_fire, "LNU")
df_lnu["year"] = df_lnu["year"].astype(str).map(lambda x: x+'/12/31')

df_la = combine_data(df_fire, "LAC")
df_la["year"] = df_la["year"].astype(str).map(lambda x: x+'/12/31')

df_vnc = combine_data(df_fire, "VNC")
df_vnc["year"] = df_vnc["year"].astype(str).map(lambda x: x+'/12/31')

df_slo = combine_data(df_fire, "SLU")
df_slo["year"] = df_slo["year"].astype(str).map(lambda x: x+'/12/31')

df_butte = combine_data(df_fire, "BTU")
df_butte["year"] = df_butte["year"].astype(str).map(lambda x: x+'/12/31')

df_river = combine_data(df_fire, "RRU")
df_river["year"] = df_river["year"].astype(str).map(lambda x: x+'/12/31')

In [53]:
# let's determine exactly what features are good for predicting fire frequency and 
# size with model selection and hyperparameter tuning (5.6)

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

def get_cv_error(df, features, prediction):
  # define pipeline
  pipeline = make_pipeline(
      StandardScaler(),
      KNeighborsRegressor(n_neighbors=5)
  )
  # calculate errors from cross-validation
  cv_errs = -cross_val_score(pipeline, X=df[features], 
                             y=df[prediction],
                             scoring="neg_mean_squared_error", cv=10)
  # calculate average of the cross-validation errors
  return cv_errs.mean()

In [55]:
# calculate and store errors for different feature sets
errs = pd.Series()
for features in [["temp_value"], 
                 ["temp_anomaly"],
                 ["precip_value"],
                 ["precip_anomaly"],
                 ['temp_value', 'temp_anomaly'],
                 ['temp_value', 'precip_value'],
                 ['temp_value', 'precip_anomaly'],
                 ['temp_anomaly', 'precip_value'],
                 ['temp_anomaly', 'precip_anomaly'],
                 ['precip_value', 'precip_anomaly'],
                 ['temp_value', 'temp_anomaly', 'precip_value'],
                 ['temp_value', 'temp_anomaly', 'precip_anomaly'],
                 ['temp_value', 'precip_value', 'precip_anomaly'],
                 ['temp_anomaly', 'precip_value', 'precip_anomaly'],
                 ["temp_value", "temp_anomaly", "precip_value", "precip_anomaly"]]:
  errs[str(features)] = get_cv_error(df_lnu, features, "frequency")

errs

# ['temp_value', 'temp_anomaly', 'precip_value'] and ['temp_value', 'temp_anomaly', 'precip_anomaly'] 
# are the two best variable combinations for predicting fire frequency (although they're not that great to be fair)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


['temp_value']                                                      79.299974
['temp_anomaly']                                                    79.302103
['precip_value']                                                    79.127282
['precip_anomaly']                                                  79.127282
['temp_value', 'temp_anomaly']                                      79.685205
['temp_value', 'precip_value']                                      76.037538
['temp_value', 'precip_anomaly']                                    76.037538
['temp_anomaly', 'precip_value']                                    76.037538
['temp_anomaly', 'precip_anomaly']                                  76.037538
['precip_value', 'precip_anomaly']                                  79.127282
['temp_value', 'temp_anomaly', 'precip_value']                      74.790487
['temp_value', 'temp_anomaly', 'precip_anomaly']                    74.790487
['temp_value', 'precip_value', 'precip_anomaly']                

In [56]:
from sklearn.model_selection import GridSearchCV

pipeline = make_pipeline(
    StandardScaler(),
    KNeighborsRegressor(n_neighbors=5)
) 

grid_search = GridSearchCV(pipeline,
                           param_grid={
                               "kneighborsregressor__n_neighbors": range(1, 20)
                           },
                           scoring="neg_mean_absolute_error",
                           cv=5)
grid_search.fit(df_lnu[['temp_value', 'precip_value', 'precip_anomaly']], df_lnu["frequency"])
grid_search.best_estimator_

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kneighborsregressor', KNeighborsRegressor(n_neighbors=13))])

In [58]:
# use the k value and categories on a knearestneighbor model, make a prediction and look at the RMSE, MAE, and R^2
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def test_models(df):
  X_train = df[['temp_value', 'precip_value', 'precip_anomaly']]
  y_train = df["frequency"]

  pipeline = make_pipeline(
            StandardScaler(),
            KNeighborsRegressor(n_neighbors=13)
  )
  pipeline.fit(X=X_train, y=y_train)

  y_train_ = pipeline.predict(X=X_train)

  rmse = np.sqrt(mean_squared_error(y_train, y_train_))
  mae = mean_absolute_error(y_train, y_train_)
  r2 = r2_score(y_train, y_train_)
  freq = df['frequency'].mean()

  return [rmse, mae, r2, freq]

test_metrics = []
for name, df in zip(['la', 'lnu', 'butte', 'river', 'slo', 'vnc'], [df_la, df_lnu, df_butte, df_river, df_slo, df_vnc]):
  test_metrics.append([name] + test_models(df))
pd.DataFrame(test_metrics, columns=['name', 'rmse', 'mae', 'r2', 'avg_frequency'])

,name,rmse,mae,r2,avg_frequency
0,la,13.408177,9.769231,0.177368,16.270492
1,lnu,7.177229,4.479823,0.190035,4.778689
2,butte,2.174520,1.511349,0.231203,1.532787
3,river,9.528668,5.421185,0.176052,6.795082
4,slo,5.927219,3.213115,0.235293,3.327869
5,vnc,5.539088,2.511349,0.239295,3.245902


In [61]:
# let's try linear regression just for the hell of it
# tried with both log(freq) and freq, log seems to have much better predictions 
# but the correlation is still horrendous
# tried with multiple county groups

def test_reg_model(df):
  df["log(frequency)"] = np.log(df["frequency"]).replace(-np.inf, 0) # ignore the warning

  X_train = df[['temp_value', 'precip_value', 'precip_anomaly']]
  y_train = df["log(frequency)"]

  model = LinearRegression()
  model.fit(X=X_train, y=y_train)
  y_train_ = model.predict(X=X_train)

  rmse = np.sqrt(mean_squared_error(y_train, y_train_))
  mae = mean_absolute_error(y_train, y_train_)
  r2 = r2_score(y_train, y_train_)

  freq = df['frequency'].mean()

  return [rmse, mae, r2, freq]

test_metrics = []
for name, df in zip(['la', 'lnu', 'butte', 'river', 'slo', 'vnc'], [df_la, df_lnu, df_butte, df_river, df_slo, df_vnc]):
  test_metrics.append([name] + test_models(df))
pd.DataFrame(test_metrics, columns=['name', 'rmse', 'mae', 'r2', 'avg_frequency'])

,name,rmse,mae,r2,avg_frequency
0,la,13.408177,9.769231,0.177368,16.270492
1,lnu,7.177229,4.479823,0.190035,4.778689
2,butte,2.174520,1.511349,0.231203,1.532787
3,river,9.528668,5.421185,0.176052,6.795082
4,slo,5.927219,3.213115,0.235293,3.327869
5,vnc,5.539088,2.511349,0.239295,3.245902
